# DrakeVisualizer.jl
This notebook demonstrates interacting with the Drake Visualizer app from Julia. On Mac and Ubuntu, just installing `DrakeVisualizer.jl` will also install a pre-built version of the standalone `drake-visualizer` executable. 

If you're not on one of those platforms, you'll need to install Director (which includes `drake-visualizer`) from <https://github.com/robotlocomotion/director>

In [1]:
# Activate the DrakeVisualizer package, and import some other 
# useful functions
using DrakeVisualizer
using CoordinateTransformations
using Interact
import GeometryTypes: HyperRectangle, Vec, HomogenousMesh
import ColorTypes: RGBA

adding: /Users/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [2]:
# Run the following command to launch the viewer application:
# proc = DrakeVisualizer.new_window()

In [3]:
# First, we'll create a simple geometric object
box = HyperRectangle(Vec(0.,0,0), Vec(1.,1,1))

GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0))

In [4]:
# Visualizer() causes the viewer to spawn a geometry or a set of geometries. 
# It returns a Visualizer, which includes all the information
# about that loaded geometry. 
# Note that the model is initially loaded in the zero configuration 
# (that is, its position and rotation are all zeros)
model = Visualizer(box)

Visualizer with robot_id_number: 1

In [5]:
# We can use draw() to tell the viewer to draw the box at a specific
# position. Translation() creates a CoordinateTransformations.Transformation
# corresponding to the given x; y; z translation.
draw(model, [Translation(1.,0,0)])

In [6]:
# We can also rotate the model by sending a different transformation
draw(model, [LinearMap(AngleAxis(pi/4, 0, 0, 1))])

In [7]:
# Let's look more at what's being done under the hood. The model
# contains an ordered dictionary from keys to Link objects. Each 
# Link is a list of GeometryData objects.
# And each GeometryData contains a single geometric primitive as
# well as information about its color and its position with
# respect to the link that contains it.
#
# Calling Visualizer() on a single geometry (the box above) results in 
# a new Robot being automatically created. That Robot has one link,
# and that Link has one GeometryData whose geometry is the box we
# provided.
model.links

DataStructures.OrderedDict{Int64,Array{DrakeVisualizer.GeometryData,1}} with 1 entry:
  1 => DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTran…

In [8]:
# Here's that single link. Since we didn't provide a key for our 
# geometry, it was automatically given the key 1:
model.links[1]

1-element Array{DrakeVisualizer.GeometryData,1}:
 DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HyperRectangle{3,Float64}}(GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0)),CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))

In [9]:
# Here's the box we created. It's the first (and only) geometry
# of the first (and only) link of the robot. 
model.links[1][1]

DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HyperRectangle{3,Float64}}(GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0)),CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))

In [10]:
@assert model.links[1][1].geometry === box

In [11]:
# Now let's make some more complicated robots. We'll create a 
# new GeometryData from the box, but color it green this time.
green_box_data = GeometryData(box)
green_box_data.color = RGBA(0., 1, 0, 0.5)
model = Visualizer(green_box_data);

In [12]:
# Now let's create a blue box
blue_box_data = GeometryData(box, Translation(0,0,1.), RGBA(0,0,1,0.5))

# We can create a new Link that contains two geometries: the 
# green box and the blue box. 
link1 = Link([green_box_data; blue_box_data])
model = Visualizer(link1);

In [13]:
# And we can draw that new link wherever we like.
draw(model, [Translation(0.75, 0, 0)])

# Note that the green and blue boxes move together. That's
# because they are part of the same rigid Link. 

In [14]:
# If we want different parts of our robot to move separately,
# they need to be on separate links. Let's create a second
# Link and then a robot containing both of those links. 
red_box_data = GeometryData(box)
link2 = Link([red_box_data])

# When we load this new robot, the two links are both drawn 
# at position [0; 0; 0]; right on top of each other.
model = Visualizer([link1, link2]);

In [15]:
# Now that we have two links, we need to specify two transforms
# for the draw() command. The order of the transforms matches
# the order of the links in "Robot([link1; link2])"
draw(model, [Translation(0.5, 0, 0), Translation(-0.5, 0, 0)])

In [16]:
# We can now move the two links independently
@manipulate for x1 in linspace(0, 2), x2 in linspace(0, 2)
    draw(model, [Translation(x1, 0, 0); Translation(x2, 0, 0)])
end

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"x1",0.9795918367346939,linspace(0.0,2.0,50),".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"x2",0.9795918367346939,linspace(0.0,2.0,50),".3f",true)

nothing

In [17]:
# Remembering the order of the links is likely to get confusing
# as our robots get more complicated. Instead, we can give each
# link a name (or any other identifying key) and then use a 
# dictionary to specify the pose of each link:

model = Visualizer(Dict(:red => red_box_data, :bluegreen => link1));

In [18]:
# Now we can set the positions of the links by passing a Dict to 
# draw():

draw(model, Dict(:red => IdentityTransformation(), 
                 :bluegreen => Translation(0., 0, 2.0)))

In [19]:
# We don't have to draw all of the links ever time. Let's just move
# the red box:
draw(model, Dict(:red => Translation(1., 0, 0)))

In [20]:
# Of course, we can draw much more interesting geometries than 
# just simple boxes. Let's load a 3D mesh and visualize it:
using MeshIO
using FileIO
cat = load(joinpath(Pkg.dir("GeometryTypes"), "test", "data", "cat.obj"))
model = Visualizer(cat);
draw(model, [LinearMap(AngleAxis(pi/2, 1, 0, 0))])

In [21]:
# Next, let's create a triangulated mesh by finding
# the 0-level set of some function. 
# 
# First, we'll define our function:
f = x -> sum(sin(5 * x))

# Then we pick a region of interest in which to sample the function.
# This region starts at (-1, -1, -1) and extends to (1, 1, 1):
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

# Those two pieces of information are all we need to construct a robot
# geometry. For this, we'll need the contour_mesh function:
mesh = contour_mesh(f, lower_bound, upper_bound)
# Under the hood, this will sample f at regularly spaced points inside
# the bounding rectangle, then compute a surface that connects all the 
# points for which f(x) = 0.

# And now we can load that geometry into the visualizer
model = Visualizer(mesh)

Visualizer with robot_id_number: 1

In [22]:
# We can even manipulate the geometry by changing the iso level. 
# By default, contour_mesh constructs a mesh connecting the points 
# in space for which f(x) = 0, where 0 is called the isosurface level
# or iso level. But we can change that iso level to any number we want:

f = x -> sum(sin(5 * x))
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

@manipulate for iso_level in linspace(-1, 1, 51)
    geometry = contour_mesh(f, lower_bound, upper_bound, iso_level)
    model = Visualizer(geometry)
end

# Note that for high iso_level values our geometry gets cut off at the
# edges. We could fix that by replacing the bounds with a bigger box. 

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"iso_level",0.0,linspace(-1.0,1.0,51),".3f",true)

Visualizer with robot_id_number: 1

In [23]:
# It is also possible to visualize multiple robots at the same time,
# while sending separate draw() commands for each robot. Internally,
# each robot is identified by it's ID number, so we just have to give
# the two robots different ID numbers:

vis1 = Visualizer(link1, 1)
vis2 = Visualizer(link2, 2)
@show vis1 vis2;

vis1 = Visualizer with robot_id_number: 1
vis2 = Visualizer with robot_id_number: 2


In [24]:
# To demonstrate this, let's move the first robot while leaving the
# second one where it is. We do that by calling draw() on vis1:
@manipulate for z in linspace(0, 2)
    draw(vis1, [Translation(0, 0, z)])
end

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"z",0.9795918367346939,linspace(0.0,2.0,50),".3f",true)

nothing

In [25]:
# Now let's move just the second robot by calling draw() on vis2:
@manipulate for x in linspace(0, 2)
    draw(vis2, [Translation(x, 0, 0)])
end

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"x",0.9795918367346939,linspace(0.0,2.0,50),".3f",true)

nothing

In [ ]:
# Clear all our robots
clear()

In [28]:
# Close the viewer:
# kill(proc)